### things we can try to experiment with:

- linkage

- connectivity measure

- affinity

In [1]:
import csv
import os
import pickle
import re
import sys
import numpy as np

from joblib import Parallel, delayed

from difflib import SequenceMatcher,get_close_matches

from scipy import spatial

from tqdm import *

module_path = os.path.abspath(os.path.join('../helpers/'))
if module_path not in sys.path:
    sys.path.append(module_path)

# my stuff in the helpers/ directory
import embeddings_helper, files_helper, texts_helper, metrics_helper, tags_helper, cobrinha_helper

from cobrinha_helper import get_metrics_for_tag_pair

In [2]:
# def get_metrics_for_tag_pair(tag_a, 
#                       tag_b, 
#                       tag_vectors_index, 
#                       pairwise_similarity_index,
#                       global_similarity_index):

In [3]:
PICKLE_ROOT = "/media/felipe/SSD_VOLUME//auto-tagger/data/tag-hierarchy/"

tag_vocabulary = pickle.load(open(PICKLE_ROOT+"tag_vocabulary.p","rb"))
tag_vectors_index = pickle.load(open(PICKLE_ROOT+"tag_vectors_index.p","rb"))
global_similarity_index = pickle.load(open(PICKLE_ROOT+"global_similarity_index.p","rb"))
sorted_pairwise_similarity_dict = pickle.load(open(PICKLE_ROOT+"sorted_pairwise_similarity_dict.p","rb"))

In [4]:
tag_vectors_index['sql']

array([ 13.31756595,   1.2150911 ,   0.30977384, ...,   0.        ,
         0.        ,   0.        ])

let's select a couple to 

In [9]:
tag_vectors_as_list = list()

for key, value in tag_vectors_index.items():
    tag_vectors_as_list.append((key,value))
   
ordered_tag_vectors = sorted(tag_vectors_as_list, key= lambda tpl: tpl[0])    

In [10]:
points = list()
labels = list()

for key,value in ordered_tag_vectors:
    points.append(value)
    labels.append(key)
    
data = np.vstack(points)

data.shape

(618, 4692)

In [12]:
labels[:10]

['.htaccess',
 '.net',
 '.net-4.0',
 '3d',
 'access-vba',
 'actionscript-3',
 'active-directory',
 'activerecord',
 'ajax',
 'algorithm']

In [11]:
from sklearn.cluster import AgglomerativeClustering
import matplotlib.pyplot as plt
from sklearn.neighbors import kneighbors_graph

In [15]:
# Generate sample data
n_samples = 1500
np.random.seed(0)
t = 1.5 * np.pi * (1 + 3 * np.random.rand(1, n_samples))
x = t * np.cos(t)
y = t * np.sin(t)


X = np.concatenate((x, y))
X += .7 * np.random.randn(2, n_samples)
X = X.T
X.shape

(1500, 2)

In [14]:
X

array([[ 13.26942176,  -2.5793499 ],
       [ -9.52436093,  10.39179655],
       [ 10.37103658,   8.4106398 ],
       ..., 
       [  6.28494345,  -0.25755083],
       [  3.73669373,  -3.71119963],
       [ -1.22031564,   8.24154775]])

In [16]:
kneighbors_graph(X, 30, include_self=False)

<1500x1500 sparse matrix of type '<class 'numpy.float64'>'
	with 45000 stored elements in Compressed Sparse Row format>

In [ ]:
# W=50
# H=20
# P=5
# TRUNCATE_MODE='level'
# LINKAGE='single'

# plt.figure(figsize=(W,H))

# Z = linkage(data,LINKAGE)

# dn = dendrogram(Z,labels=labels,truncate_mode=TRUNCATE_MODE,p=P,leaf_font_size=40)

# fig = plt.gcf()

# ax = plt.gca()

# ax.tick_params(axis='x', which='major', labelsize=40)
# ax.tick_params(axis='y', which='major', labelsize=40)

# fig.savefig("dendrograms/{}-{}-{}.svg".format(LINKAGE,TRUNCATE_MODE,P), format='svg')


analysis only

In [ ]:
Z[7]

In [ ]:
dn2 = dendrogram(Z,labels=labels)

In [ ]:
dn2["leaves"]